In [1]:
import pandas as pd
import os
df=pd.read_excel("Data_For Application.xlsx")

In [ ]:
import json
import numpy as np

from bert4keras.backend import keras, search_layer, K
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from keras.layers import Lambda, Dense,Bidirectional,LSTM
from keras.losses import kullback_leibler_divergence as kld
from tqdm import tqdm
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'


In [ ]:
df["text"]=df["Text"]

In [ ]:
import re
import jieba_fast as jieba
ting=[i.strip() for i in open("stop_words_ch-停用词表.txt",encoding='gbk').readlines()]
def fenci(text):
    return [i for i in jieba.cut(str(text)) if i not in ting] 

df["text"]=df["text"].apply(lambda x:fenci(x))

In [ ]:
df["text"]=df["text"].apply(lambda x:" ".join(x))

In [ ]:
import pandas as pd
num_classes = 2
maxlen = 64

# BERT base
config_path = 'chinese_roberta_www_ext/bert_config.json'
checkpoint_path = 'chinese_roberta_www_ext/bert_model.ckpt'
dict_path = 'chinese_roberta_www_ext/vocab.txt'

from tqdm import tqdm

In [ ]:
df=df.dropna(subset=["text"])
df=df["text"]
# 加载数据集
test_data=df

In [ ]:
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

class data_generator_for_test(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids = [], []
        for is_end, text in self.sample(random):
#             print(text,text1,label)
            token_ids, segment_ids = tokenizer.encode(text, maxlen=50)
            for i in range(1):
                batch_token_ids.append(token_ids)
                batch_segment_ids.append(segment_ids)
            if len(batch_token_ids) == self.batch_size  or is_end:
                batch_token_ids = sequence_padding(batch_token_ids,length=50)
                batch_segment_ids = sequence_padding(batch_segment_ids,length=50)
                yield [batch_token_ids, batch_segment_ids]
                batch_token_ids, batch_segment_ids = [], []

In [ ]:
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    return_keras_model=False,
)
tmp=bert.model.output
# tmp=bert.model.output

In [ ]:
# lstmout = Bidirectional(LSTM(128, return_sequences=False))(tmp)
lstmout = Bidirectional(LSTM(128, return_sequences=False))(tmp)
output = Dense(
    units=2,
    activation='softmax',
    kernel_initializer=bert.initializer
)(lstmout)

In [ ]:
model = keras.models.Model(bert.model.input, output)
model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=Adam(2e-5),
    metrics=['sparse_categorical_accuracy']
)
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total


class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(test_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('bertlstm.weights'.format(1))
        print(
            u'val_acc: %.5f, best_val_acc: %.5f\n' %
            (val_acc, self.best_val_acc)
        )




In [ ]:
from tqdm import tqdm
model.load_weights("bertlstm.weights")
test_generator = data_generator_for_test(test_data, 24)
preall=[]
for x_true in tqdm(test_generator):
    y_pred = model.predict(x_true)
    preall.extend(y_pred)

df_result = pd.DataFrame({"text":df,"pred_0": np.array(preall)[:,0],"pred_1":np.array(preall)[:,1]})
df_result.to_csv('test_result_application.csv', index=False)